In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import openai
import json
import ijson
import os

openai.api_key = os.getenv('KEY')

openai.base_url = "https://api.vsegpt.ru:6070/v1/"

In [2]:
openai.chat

In [ ]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 756
    true_answers = 483
    answers = []
    for j, item in enumerate(array_items):
        if j < 756:
            continue
        messages = []
        i += 1

        question = item["question_eng"]
        try:
            answer = ""
            for x in item["answers"][0]["wd_names"]["en"]:
                answer += x.lower() + ' '
        except:
            continue
        prompt = "Answer shortly in 1-3 words. If answer is a number, print digits"
        # prompt = "Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже, при этом не забывай о пунктуации."
        # messages.append({"role": "user", "content": prompt})
        messages.append({"role": "user", "content": prompt + ' ' + question})
        response = ""
        try:
            response_big = openai.chat.completions.create(
                model="openai/gpt-3.5-turbo-1106",
                messages=messages,
                temperature=0.1,
            )
        except:
            continue
        response = response_big.choices[0].message.content.lower()
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer
        })
        if response in answer:
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers, indent=4)
        with open("questions_answers_rubq.json", "w") as json_file:
            json_file.write(json_data)

In [3]:
df_test = pd.read_csv('archive/n_annotated_wd_data_test_answerable.csv')
df_test.head()

df_train = pd.read_csv('archive/n_annotated_wd_data_train_answerable.csv')

df_valid = pd.read_csv('archive/n_annotated_wd_data_valid_answerable.csv')


In [4]:
df_test['q'][1223]

'Name a person born in queens\n'

In [ ]:
# print(df_train.loc[0])
# print(df_train['q'][0], df_train['e1'][0])
i = 0
true_answers = 0
answers = []
for question, answer in zip(df_test['q'], df_test['e2']):
    if i == 201:
        break
    messages = []
    i += 1

    prompt = "Answer shortly in 1-3 words"
    # prompt = "Отвечай наиболее коротко, 1-3 слова"
    # prompt = "Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже, при этом не забывай о пунктуации."
    # messages.append({"role": "user", "content": prompt})
    messages.append({"role": "user", "content": prompt + ' ' + question})
    response = ""
    try:
        response_big = openai.chat.completions.create(
            model="openai/gpt-3.5-turbo-1106",
            messages=messages,
            temperature=0.1,
        )
        response = response_big.choices[0].message.content.lower()
    except Exception as e:
        print(e)
        response = "Error"
    answers.append({
        "question": question,
        "answer": response,
        "correct_answer": answer.lower(),
        "is_correct": answer.lower() in response
    })
    if answer.lower() in response:
        true_answers += 1
    print(i, true_answers)
    json_data = json.dumps(answers, indent=4)
    with open("questions_answers_tmp.json", "w") as json_file:
        json_file.write(json_data)

In [ ]:
len(df_test['q'])

5621